In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from itertools import combinations
from operator import add
import math
import time
import sys

In [2]:
startTime = time.time()

case_num = int(sys.argv[1])#2
global_threshold = int(sys.argv[2])#9
input_file_path = sys.argv[3]#r'C:\Users\11921\OneDrive\FilesTransfer\DSCI 553\Assignment\Assignment2\small2.csv'
output_file_path = sys.argv[4]#'output.txt'

sc = SparkContext.getOrCreate()

In [3]:
partition_num = 2

def turnStr2Pair(pairStr, case):
    if case == 1:
        return (pairStr.split(',')[0], pairStr.split(',')[1])
    else:
        return (pairStr.split(',')[1], pairStr.split(',')[0])

# read csv file and generate pairs
with open(input_file_path) as f:
    rawStrList = f.readlines()[1:]
    pairList = [pair.split('\n')[0] for pair in rawStrList]
    f.close()

if case_num == 1:    
    MBModel_RDD = sc.parallelize(pairList, partition_num).map(lambda pairStr:turnStr2Pair(pairStr, 1))\
                                      .groupByKey().mapValues(lambda iterable:sorted(list(iterable)))\
                                      .map(lambda pair: pair[1])
elif case_num == 2:
    MBModel_RDD = sc.parallelize(pairList, partition_num).map(lambda pairStr:turnStr2Pair(pairStr, 2))\
                                      .groupByKey().mapValues(lambda iterable:sorted(list(iterable)))\
                                      .map(lambda pair: pair[1])


In [5]:
def generateFrequentSingletons(chunk_list, local_threshold):
    counter_dict = dict()
    frequent_singletons = list()   
    candidate = list()
    
    for basket in chunk_list:
        for item in basket:
            if item not in counter_dict.keys():
                counter_dict[item] = 1
            else:
                counter_dict[item] += 1
    candidate = counter_dict.keys()
    for key, value in counter_dict.items():
        if value >= local_threshold: 
            frequent_singletons.append(key)
    return frequent_singletons, sorted(candidate)

def countFrequentItemsets(candidate_list, chunk_list, threshold):
    # candidate_list is list of tuple
    counter_dict = dict()
    frequentItemsets = list()
    
    for candidate in candidate_list:
        for basket in chunk_list:
            if set(candidate).issubset(basket):
                if candidate in counter_dict.keys():
                    counter_dict[candidate] += 1
                else:
                    counter_dict[candidate] = 1
    for itemset, count in counter_dict.items():
        if count >= threshold:
            frequentItemsets.append(tuple(sorted(itemset)))
    return frequentItemsets # list of tuple

def generateKTupleItemsets(frequent_list, k):
    
    resultKTupleList = list()
    candidateKTupleList = list()
    frequent_list = sorted(frequent_list)
    cand_pair_dict = dict()
    i = 0
    for i in range(len(frequent_list)):
        item1 = frequent_list[i]
        j = i+1
        while j <len(frequent_list):
            item2 = frequent_list[j]
            kTuple = set(item1 + item2)
            if len(kTuple)==k:
                sortedKTuple = tuple(sorted(kTuple))
                if sortedKTuple in cand_pair_dict.keys():
                    cand_pair_dict[sortedKTuple].append(item1)
                    cand_pair_dict[sortedKTuple].append(item2)
                    cand_pair_dict[sortedKTuple] = sorted(set(cand_pair_dict[sortedKTuple]))
                else:
                    cand_pair_dict[sortedKTuple] = [item1, item2]
                j+=1
                candidateKTupleList.append(sortedKTuple)
            else:
                break
    candidateKTupleList = list(set(candidateKTupleList)) # remove duplicate
    
    for candidateKTuple in candidateKTupleList:
        containsAll = True
        for element in candidateKTuple:
            if containsAll: 
                necessaryTuple = set(candidateKTuple) - set([element])
                sortedNecessaryTuple = tuple(sorted(necessaryTuple))
                if sortedNecessaryTuple not in cand_pair_dict[candidateKTuple]:
                    containsAll = containsAll & (sortedNecessaryTuple in frequent_list)
                else:
                    pass
            else:
                pass
        if containsAll:
            resultKTupleList.append(candidateKTuple)


    return sorted(resultKTupleList)

def apriori(chunk, threshold, full_size):    
    # chunk_list is needed here because chunk is of type TraversableOnce, which will be empty after calling list
    chunk_list = list(chunk) 
    chunk_size = len(chunk_list)
    # determine local(chunk) threshold, 0.9 is the scaling factor used to reduce false positives
    local_threshold = (chunk_size/full_size) * threshold
    if local_threshold < 1:
        local_threshold = 1
    
    result_candidate_itemsets = list() # contains all k-tuple itemsets as result
    true_frequent_k_itemset_list = list() # contains the true frequent itemsets used to generate candidate for next k in apriori
    next_k_candidate_list = list() # contains candidate of next k-tuple itemsets, used to decide if there are any candidate left 
    k_index = 1
    
    # generate true frequent singletons
    # true_frequent_k_itemset_list is now the frequent singletons
    # next_k_candidate_list is now the singleton candidates
    
    true_frequent_k_itemset_list, next_k_candidate_list = generateFrequentSingletons(chunk_list, local_threshold)
    
    while len(next_k_candidate_list)!=0: 
        k_index += 1 #generate 2-tuple candidates, 3-tuple, 4-tuple, etc 
        
        if k_index==2: 
            result_candidate_itemsets.append([(single,) for single in next_k_candidate_list]) #append candidate k
            next_k_candidate_list = generateKTupleItemsets([(single,) for single in true_frequent_k_itemset_list], 2) # true k-1 to candidate k  
        else:
            result_candidate_itemsets.append(next_k_candidate_list) #append candidate k
            true_frequent_k_itemset_list = countFrequentItemsets(next_k_candidate_list, chunk_list, local_threshold) #cand k to true k
                
            next_k_candidate_list = generateKTupleItemsets(true_frequent_k_itemset_list, k_index) # true k to candidate k+1
    
    yield result_candidate_itemsets

In [6]:
# mapPartition() is equivalent to first Map in SON algorithm

full_size = MBModel_RDD.count()

rawCandidates = MBModel_RDD.mapPartitions(lambda partition:apriori(partition, global_threshold, full_size))
candidatesResult = rawCandidates.flatMap(lambda x:x).flatMap(lambda x:x)\
                                .distinct().sortBy(lambda pairs: (len(pairs), pairs))\
                                .collect()

In [7]:
def countItemsets(chunk, candidates):
    resultList = list()
    chunkList = list(chunk)
    for basket in chunkList:
        for candidateItemset in candidates:
            if set(candidateItemset).issubset(set(basket)):
                resultList.append((candidateItemset,1))
    return resultList

#count occurence of the itemset in each chunk and sum them up
frequentItemsets = MBModel_RDD.mapPartitions(lambda partition: countItemsets(partition, candidatesResult)).reduceByKey(add)\
                              .filter(lambda itemset: itemset[1]>=global_threshold)\
                              .sortBy(lambda pair: (len(pair[0]), pair[0]))\
                              .map(lambda pair: pair[0])

frequentItemsetsResult = frequentItemsets.collect()

In [8]:
outFile = open(output_file_path, 'w')
outFile.write('Candidates: \n')
x_tuple_candidates_str = ''
x_length = 1
current_index = 0
for candidate in candidatesResult:
    if x_length == len(candidate):
        if x_length==1:
            x_tuple_candidates_str += '(\'' + candidate[0] + '\'),'
        else:
            x_tuple_candidates_str += str(candidate) + ','
    else: 
        x_tuple_candidates_str = x_tuple_candidates_str[:-1]
        x_tuple_candidates_str += '\n\n'
        outFile.write(x_tuple_candidates_str)
        x_tuple_candidates_str = str(candidate) + ',' 
        x_length += 1
    current_index += 1
    if current_index==len(candidatesResult): #or at EOF
        x_tuple_candidates_str = x_tuple_candidates_str[:-1]
        outFile.write(x_tuple_candidates_str)
        
outFile.write('\n\nFrequent Itemsets: \n')
x_tuple_frequent_str = ''
x_length = 1
current_index = 0
for frequentItemset in frequentItemsetsResult:
    if x_length == len(frequentItemset):
        if x_length==1:
            x_tuple_frequent_str += '(\'' + frequentItemset[0] + '\'),'
        else:
            x_tuple_frequent_str += str(frequentItemset) + ','
    else: 
        x_tuple_frequent_str = x_tuple_frequent_str[:-1]
        x_tuple_frequent_str += '\n\n'
        outFile.write(x_tuple_frequent_str)
        x_tuple_frequent_str = str(frequentItemset) + ',' 
        x_length += 1
    current_index += 1
    if current_index==len(frequentItemsetsResult): #or at EOF
        x_tuple_frequent_str = x_tuple_frequent_str[:-1]
        outFile.write(x_tuple_frequent_str)
    
outFile.close()

print("Duration: %d" % (time.time() - startTime))

Duration: 29s.
